In [1]:
# pytorch-0.3.1
# conda install pytorch=0.3.1 cuda80 -c pytorch

# cd data
# wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# tar -xzf aclImdb_v1.tar.gz
# cd ..

# mkdir data/aclImdb/models
# mkdir data/aclImdb/train/all/
# mkdir data/aclImdb/test/all/
# ln -sf ~/fastai/old/fastai/ fastai
# conda install spacy
# python -m spacy download en

In [15]:
# import os
# import shutil

# src = 'data/aclImdb/train/neg/'
# dest = 'data/aclImdb/train/all/neg'

# for (src, dest) in [
#     ('data/aclImdb/train/pos/', 'data/aclImdb/train/all/'),
#     ('data/aclImdb/train/neg/', 'data/aclImdb/train/all/'),
#     ('data/aclImdb/test/pos/', 'data/aclImdb/test/all/'),
#     ('data/aclImdb/test/neg/', 'data/aclImdb/test/all/')
# ]:
#     src_files = os.listdir(src)[:100]
#     for file_name in src_files:
#         full_file_name = os.path.join(src, file_name)
#         if (os.path.isfile(full_file_name)):
#             shutil.copy(full_file_name, dest)

In [14]:
# !rm -rf data/aclImdb/train/all/*
# !rm -rf data/aclImdb/test/all/*

In [16]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

In [17]:
PATH='data/aclImdb/'

TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

imdbEr.txt  imdb.vocab  models/  README  test/  tmp/  train/


In [18]:
bs=64; bptt=70
spacy_tok = spacy.load('en')
TEXT = data.Field(lower=True, tokenize="spacy")
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [19]:
# save TEXT
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [20]:
# batches;  # unique tokens in the vocab; # tokens in the training set; # sentences
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(11, 578, 1, 54689)

In [21]:
len(md.trn_ds)

1

In [22]:
# torchtext will handle turning this words into integer IDs for us automatically. 
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
   12
  188
   29
  514
   37
    2
    0
   64
   13
   20
    4
    0
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

In [10]:
seq2seq_reg

<function fastai.lm_rnn.seq2seq_reg>

In [11]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [13]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

A Jupyter Widget

  0%|          | 0/11 [00:00<?, ?it/s]


AssertionError: 